In [2]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.datasets import make_classification
import matplotlib.pyplot as plt
import math
%cd /content/drive/MyDrive/Dataset for ML
!ls
data=pd.read_csv('play_tennis.csv')
data

/content/drive/MyDrive/Dataset for ML
play_tennis.csv


Outlook Temperature Humidity    Wind Play
0      Sunny         Hot     High    Weak   No
1      Sunny         Hot     High  Strong   No
2   Overcast         Hot     High    Weak  Yes
3       Rain         Mid     High    Weak  Yes
4       Rain        Cool   Normal    Weak  Yes
5       Rain        Cool   Normal  Strong   No
6   Overcast        Cool   Normal  Strong  Yes
7      Sunny         Mid     High    Weak   No
8      Sunny        Cool   Normal    Weak  Yes
9       Rain         Mid   Normal    Weak  Yes
10     Sunny         Mid   Normal  Strong  Yes
11  Overcast         Mid     High  Strong  Yes
12  Overcast         Hot   Normal    Weak  Yes
13      Rain         Mid     High  Strong   No

1.Implement các hàm thống kê


In [3]:
def mean(X):
  return [sum(X[:,i])/(X.shape[0]) for i in range(X.shape[1])]
 
def median(X):
  median=[]
  for i in range(X.shape[1]):
    X[:,i].sort()
    if X.shape[0] % 2 != 0:
        median.append(X[:,i][int(X.shape[0]/2)])
    else:
        median.append(1/2*(int(X[:,i][int(X.shape[0]/2)]+X[:,i][(int(X.shape[0]/2))-1])))
  return median      
def mode(X):
  mode=[]
  for i in range(X.shape[1]):
   set_x=set(X[:,i])
   cnt=[list(X[:,i]).count(k) for k in set_x]
   most=max(cnt)
   mode.append([n for n in set_x if list(X[:,i]).count(n)==most])
  return mode
def variance(X):
  variance=[]
  X_mean=mean(X)
  for i in range(X.shape[1]):
   x_mean=X_mean[i]
   variance.append(sum([(int(X[:,i][k])-int(x_mean))**2 for k in range(X.shape[0])])/X.shape[0])
  return variance
def std(X):
  std=[]
  for i in variance(X):
    std.append(math.sqrt(i))
  return std  


2.Tạo bộ dữ liệu và tóm tắt đặc điểm của dữ liệu bằng các hàm thống kê đã khởi tạo


In [4]:
X,y=make_classification(n_samples=1000,n_classes=5,n_features=10,n_informative=10,n_redundant=0,n_repeated=0)
X.shape

(1000, 10)

In [5]:
print('mean(X):',mean(X))
print('median(X):',median(X))
print('mode(X):',mode(X))
print('variance(X):',variance(X))
print('std(X):',std(X))
print('/n/n')

mean(X): [-0.05962059299510555, 0.03677696611439674, -0.2269759096076491, 0.04579254695635757, -0.003059671326099368, 0.6178124986495906, -0.22277267985348825, 0.06788310340751035, 0.23218667604981208, 0.4775630556020327]
median(X): [0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.5]
mode(X): [[-0.9600052790530986, -0.9552857423404486, -0.910228019093485, -0.7905473130414666, -0.785452254200993, -0.7471982623522297, -0.7452542077143427, -0.7012444316417445, -0.6736147389855489, -0.6619565246658476, -0.42517351847772034, 2.0222503275704167, 2.4136336714217066, 2.7345794097258533, 5.627681656252331, 6.151225191256615, 7.780120341586888, -0.781803078765513, -0.09663701101573685, -0.05009625291450681, 4.5801971684457214, -0.579206345795384, -0.05960168755122153, 0.04213077338981508, 0.9119698505645464, 0.995154141789087, 1.6939312745183632, 2.20306453382797, 2.2281576171819926, 0.046433258639390074, 0.4332288399291506, 0.9147086534238518, -2.994101922565675, -2.660085989321799, -0.526591884

3.Sử dụng data Play Tennis,xây dựng và tính 1 số xác suất

In [6]:
def Probability(data):
  n=len(data[data.Play=='Yes'])
  print('P(Outlook=Overcast,Temperature=Hot/Yes)=',len(data[(data.Outlook=='Overcast')&(data.Temperature=='Hot')])/n)
  print('P(Outlook=Rain,Humidity=High/Yes)=',len(data[(data.Outlook=='Rain')&(data.Humidity=='High')])/n)
  print('P(Humidity=Normal,Temperature=Hot/Yes)=',len(data[(data.Humidity=='Normal')&(data.Temperature=='Hot')])/n)
  print('P(Temperature=Mid,Wind=Strong/Yes)=',len(data[(data.Temperature=='Mid')&(data.Wind=='Strong')])/n)      
Probability(data)

P(Outlook=Overcast,Temperature=Hot/Yes)= 0.2222222222222222
P(Outlook=Rain,Humidity=High/Yes)= 0.2222222222222222
P(Humidity=Normal,Temperature=Hot/Yes)= 0.1111111111111111
P(Temperature=Mid,Wind=Strong/Yes)= 0.3333333333333333


4.Implement Naive Bayes Classifier

In [7]:
from typing import ValuesView
class Naive_Bayes_Classifier():
  def __init__(self):
    pass
  def prob(self,data):
    self.n_classes=len(set(data.iloc[:,-1].values))
    N=data.shape[1]
    all_prob_given=dict()
    all_prob_class=dict()
    for cls in set(data.iloc[:,-1].values):
      all_prob_class[str(cls)]=len(data[data.iloc[:,-1]==cls])/data.shape[0]
      prob_given_class=[]
      prob_feature=[]
      for i in range(N-1):
        prob_value=[]
        value_feature=set(data.iloc[:,i].values)
        n=len(value_feature)
        for k in value_feature:
          prob_value.append([k,(len(data[(data.iloc[:,i]==k)&(data.iloc[:,-1]==cls)])+1)/(len(data[data.iloc[:,-1]==cls])+n)])
        prob_feature.append(prob_value)  
      prob_given_class.append(prob_feature)
      all_prob_given[str(cls)]=prob_given_class
    self.all_prob_given=all_prob_given
    self.all_prob_class=all_prob_class 
    return self.all_prob_given,self.all_prob_class,self.n_classes
  def predict(self,X):
    prob_pred=np.ones((X.shape[0],self.n_classes))
    for i in range(X.shape[0]):
      x=X.iloc[i,:].values
      j=0
      for key in self.all_prob_given:
        for k in range(X.shape[1]):
          for n in self.all_prob_given[str(key)][k]:
            if x[k]==n[0]:
              prob_pred[i,j]*=n[1]
              break
        pro_pred[i,j]*=self.all_prob_class[str(key)]      
        j+=1  
      pro_pred[i]=(pro_pred[i])/sum(pro_pred[i])        
    return pro_pred
    


 

In [8]:
model=Naive_Bayes_Classifier()
proba=model.prob(data)
proba

({'No': [[[['Rain', 0.375], ['Overcast', 0.125], ['Sunny', 0.5]],
    [['Mid', 0.375], ['Cool', 0.25], ['Hot', 0.375]],
    [['Normal', 0.2857142857142857], ['High', 0.7142857142857143]],
    [['Strong', 0.5714285714285714], ['Weak', 0.42857142857142855]]]],
  'Yes': [[[['Rain', 0.3333333333333333],
     ['Overcast', 0.4166666666666667],
     ['Sunny', 0.25]],
    [['Mid', 0.4166666666666667], ['Cool', 0.3333333333333333], ['Hot', 0.25]],
    [['Normal', 0.6363636363636364], ['High', 0.36363636363636365]],
    [['Strong', 0.36363636363636365], ['Weak', 0.6363636363636364]]]]},
 {'No': 0.35714285714285715, 'Yes': 0.6428571428571429},
 2)